## Stimulus Function Tables and External Resources in NWB

To prototype the inclusion of stimulus metadata in NWB files using the Estim ontology, we created an extension to the NWB schema for a stimulus function table that holds waveforms and their parameters, and show how this links to the new external resources table.

In [1]:
import datetime
import inspect
import numpy as np
import pandas as pd
import os

from pynwb import NWBHDF5IO, NWBFile
from pynwb.core import DynamicTableRegion, DynamicTable
from pynwb.device import Device

from pynwb import register_class, load_namespaces
from hdmf.utils import docval, call_docval_func, getargs, get_docval



# namespace for the SFT extension
ndx_stimulation_specpath = '/Users/pam.baker/Documents/ndx_stim/ndx-stimulation/spec/ndx-stimulation.namespace.yaml'

# Load the namespace                                                                                                 
load_namespaces(ndx_stimulation_specpath)

{'ndx-stimulation': {'core': ('Container',
   'Data',
   'DynamicTable',
   'ElementIdentifiers',
   'VectorData'),
  'hdmf-common': ('Data', 'VectorData', 'VectorIndex')}}

In [2]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

The stimulus function table is designed to hold the names of stimulus waveforms and their relevant parameters - float parameters for numeric values and function parameters that reference other waveform functions (for parameter values that vary as a function of time).

In [3]:
@register_class('StimulusFunctionTable', 'ndx-stimulation')
class StimulusFunctionTable(DynamicTable):   
    
    __columns__ = ( 
                    {'name':'function_name', 
                     'description': 'The names of the 1D stimulus waveforms.',
                     'required': True,
                     'index': False},
                    {'name':'float_parameters', 
                     'description': 'The names of the float parameters for the 1D stimulus waveforms.',
                     'required': True,
                     'index': True},
                    {'name':'function_parameters',
                     'description': 'The function parameters for the 1D stimulus waveforms.',
                     'required': True,
                     'index': True}
    )
    
    
    @docval(*get_docval(DynamicTable.__init__, 'id', 'columns', 'colnames'))
    def __init__(self, **kwargs):
        kwargs['name'] = ('StimulusFunctionTable')
        kwargs['description'] = ('Table for storing ontologized 1D stimulus waveform metadata')
        call_docval_func(super().__init__, kwargs)
        

We create an example stimulus function table to hold a couple of square waves with associated parameters. We use the add_row function inherited from Dynamic Tables to add new entries.

In [4]:
sft = StimulusFunctionTable()

float_params_sq1 = [('amplitude', -0.110, 'V'), 
                    ('duration', 0.500, 's'),
                    ('start_time', 0.1, 's')
                   ]

float_params_sq2 = [('amplitude', 0.090, 'V'), 
                    ('duration', 1, 's'),
                    ('start_time', 0.1, 's')
                   ]

float_params_ramp = [('amplitude', 0.200, 'V'), 
                     ('duration', 1, 's'),
                     ('start_time', 0.05, 's')
                    ]

float_params_sin = [('amplitude', 0.090, 'V'), 
                    ('duration', 1, 's'),
                    ('start_time', 0.1, 's')
                   ]

func_params_sin = [('frequency', 2, 'Hz')]


wave1 = {'function_name':'sq', 
         'float_parameters': float_params_sq1, 
         'function_parameters':[]}

wave2 = {'function_name':'sq', 
         'float_parameters': float_params_sq2, 
         'function_parameters':[]}

wave3 = {'function_name':'ramp', 
         'float_parameters': float_params_ramp, 
         'function_parameters':[]}

wave4 = {'function_name':'sin', 
         'float_parameters': float_params_sin, 
         'function_parameters': func_params_sin}


# Using add_row from DynamicTable

sft.add_row(data = wave1)
sft.add_row(data = wave2)
sft.add_row(data = wave3)
sft.add_row(data = wave4)


pd.set_option("display.max_colwidth", 100)

display(sft.to_dataframe())

,function_name,float_parameters,function_parameters
id,,,
0,sq,"[(amplitude, -0.11, V), (duration, 0.5, s), (start_time, 0.1, s)]",[]
1,sq,"[(amplitude, 0.09, V), (duration, 1, s), (start_time, 0.1, s)]",[]
2,ramp,"[(amplitude, 0.2, V), (duration, 1, s), (start_time, 0.05, s)]",[]
3,sin,"[(amplitude, 0.09, V), (duration, 1, s), (start_time, 0.1, s)]","[(frequency, 2, Hz)]"


## External Resources Table

- Resources: eg ontology we are referencing
- Entities: as defined by an entry in ontology/controlled terms
- Keys: reference from object into the ER tables that maps onto entities 
- Objects: thing we are ontologizing (eg SFT)


In [5]:
from hdmf.common import ExternalResources
from hdmf import Container, Data
import pandas as pd

er = ExternalResources(name='ExtResTable')

/opt/anaconda3/envs/pynwb_prerel/lib/python3.8/site-packages/hdmf/container.py:181: UserWarning: ExternalResources is experimental -- it may be removed in the future and is not guaranteed to maintain backward compatibility
  warn(_exp_warn_msg(cls))


The *StimulusFunctionTable* is the container object we will link to in the External Resources

In [6]:
object_id = sft.object_id
print(object_id)

5dc3b277-ac95-4e09-b2ff-dfad53ac8351


In [9]:
# add_ref creates an entry across all tables (entities, keys, objects, resources)

er.add_ref(container = object_id, 
           field='', 
           key='sq',
           resource_name='Estim Ontology',
           resource_uri='Estim_Onto_uri',
           entity_id='Estim_square_ID',
           entity_uri='Estim_square_uri'
          )

er.add_ref(container = object_id, 
           field='', 
           key='duration',
           resource_name='Estim Ontology',
           resource_uri='Estim_Onto_uri',
           entity_id='Estim_duration_ID',
           entity_uri='Estim_duration_uri'
          )

er.add_ref(container = object_id, 
           field='', 
           key='amplitude',
           resource_name='Estim Ontology',
           resource_uri='Estim_Onto_uri',
           entity_id='Estim_amplitude_ID',
           entity_uri='Estim_amplitude_uri'
          )

er.add_ref(container = object_id, 
           field='', 
           key='start_time',
           resource_name='Estim Ontology',
           resource_uri='Estim_Onto_uri',
           entity_id='Estim_shift_ID',
           entity_uri='Estim_shift_uri'
          )


(<hdmf.common.resources.Key at 0x19f557e50>,
 <hdmf.common.resources.Entity at 0x19f557ee0>)

In [10]:
er.resources.to_dataframe()

,resource,resource_uri
0,Estim Ontology,Estim_Onto_uri


In [11]:
er.keys.to_dataframe()

,key
0,sq
1,duration
2,amplitude
3,start_time


In [12]:
er.entities.to_dataframe()

,keys_idx,resources_idx,entity_id,entity_uri
0,0,0,Estim_square_ID,Estim_square_uri
1,1,0,Estim_duration_ID,Estim_duration_uri
2,2,0,Estim_amplitude_ID,Estim_amplitude_uri
3,3,0,Estim_shift_ID,Estim_shift_uri


In [ ]:
k=er.add_key(key_name='sq')
er.keys.to_dataframe()

In [ ]:
key_object = er.get_key(key_name='sq')
print(key_object)

### Write

In [13]:
from datetime import datetime
from dateutil.tz import tzlocal

start_time = datetime(2017, 4, 3, 11, tzinfo=tzlocal())
create_date = datetime(2017, 4, 15, 12, tzinfo=tzlocal())


nwbfile = NWBFile('demo', 'NWB456', start_time,
                  file_create_date=create_date)

nwbfile.add_acquisition(sft)

In [14]:
# Write the SFT out to file

from pynwb import NWBHDF5IO

io = NWBHDF5IO('sft_ert_testy.nwb', mode='w')
io.write(nwbfile)
io.close()

In [16]:
# Reading in the file I just wrote

io = NWBHDF5IO('sft_ert_testy.nwb', mode='r', load_namespaces=True)
sft_nwbfile = io.read()

In [ ]:
# print(sft_nwbfile.acquisition)

sft_in = sft_nwbfile.get_acquisition()

In [ ]:
sft_in.to_dataframe()

In [ ]:
sft_in['function_parameters'].target[0]